In [1]:
FIGURES_PATH = 'out/figures/'
DATASETS_PATH = 'out/datasets/'
DICTS_PATH = 'out/dicts/'
CLUSTERS_PATH = 'out/clusters/'

In [2]:
import pandas as pd
from datetime import datetime, timedelta
import os
import multiprocessing
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import random
from tqdm.notebook import tqdm
from multiprocesspandas import applyparallel
from pandarallel import pandarallel
import psutil
from sys import getsizeof
import networkx as nx
from scipy.cluster.hierarchy import linkage, fcluster


from netgraph import Graph, InteractiveGraph, EditableGraph

import pickle
import gc 


tqdm.pandas()
from helper import *

In [3]:
NROWS = None

In [4]:
data = pd.read_csv(DATASETS_PATH + 'data_processed' + '.csv')
data.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20662675 entries, 0 to 20662674
Data columns (total 12 columns):
 #   Column           Dtype  
---  ------           -----  
 0   Unnamed: 0       int64  
 1   gid              int64  
 2   transaction_key  int64  
 3   store_id         int64  
 4   product_id       int64  
 5   line_item_price  float64
 6   line_item_cost   float64
 7   line_type        int64  
 8   datetime         object 
 9   category_id      float64
 10  weekday          int64  
 11  line_quantity    float64
dtypes: float64(4), int64(7), object(1)
memory usage: 3.2 GB


In [5]:
print(data[:1_000_000].info(memory_usage='deep'))


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 12 columns):
 #   Column           Non-Null Count    Dtype  
---  ------           --------------    -----  
 0   Unnamed: 0       1000000 non-null  int64  
 1   gid              1000000 non-null  int64  
 2   transaction_key  1000000 non-null  int64  
 3   store_id         1000000 non-null  int64  
 4   product_id       1000000 non-null  int64  
 5   line_item_price  1000000 non-null  float64
 6   line_item_cost   932885 non-null   float64
 7   line_type        1000000 non-null  int64  
 8   datetime         1000000 non-null  object 
 9   category_id      929105 non-null   float64
 10  weekday          1000000 non-null  int64  
 11  line_quantity    1000000 non-null  float64
dtypes: float64(4), int64(7), object(1)
memory usage: 156.4 MB
None


In [4]:
data = pd.read_csv('datasets/чеки.csv', nrows=NROWS)
data = data.rename(columns = {'line_item_id': 'product_id'})

In [5]:
data = data[['line_article_id', 'line_article_description']].drop_duplicates()
product_names = data.set_index('line_article_id').to_dict()['line_article_description']


In [6]:
with open(DICTS_PATH + 'products.json') as json_file:
    products_dict = json.load(json_file)
    
with open(CLUSTERS_PATH + 'dists.pkl', 'rb') as f:
    dists = pickle.load(f)

In [19]:
def concat_dicts(dict1, dict2):
    ans = dict()
    for k1 in dict1.keys():
        if int(k1) in dict2:
            ans[dict1[k1]] = dict2[int(k1)]
        else:
            ans[dict1[k1]] = 'Unnamed'
        
    return ans


In [20]:
names = concat_dicts(products_dict, product_names)
names

{0: 'Стремянка стальная Zalger с ковриком 4 ступени',
 1: 'Шпаклёвка полимерная суперфинишная Knauf Ротбанд Паста 18 кг',
 2: 'ТРОЙНИК АКС PROAQUA 20X20X20',
 3: 'Гидроизоляция без битума Knauf Флэхендихт, 5 кг',
 4: 'НАПРЕССОВОЧНАЯ ГИЛЬЗА РОСТЕРМPPSU 20Х2.8',
 5: 'Клей для плитки усиленный Knauf Флизен Плюс, 25 кг',
 6: 'БУР SDS PLUS 14 Х 460ММ DEXTER',
 7: 'Бетонконтакт Боларс 12 кг',
 8: 'Угол перфорированный ПВХ с сеткой 100х150x3000 мм',
 9: 'Заглушка Ø 50 мм полипропилен',
 10: 'Кисть для радиатора 35 мм',
 11: 'Кисть для водных красок Dexter 30 мм',
 12: 'ТРУБА REHAU RAUTITAN STABIL 20Х2.9ММ 1М',
 13: 'Рулон шлифовальный Dexter P220, 115х2500 мм, ткань',
 14: 'Unnamed',
 15: 'Отвод Ø 50х87° полипропилен',
 16: 'Пескобетон Axton 30 кг',
 17: 'Ножницы для труб Dexter',
 18: 'Пробка Equation, наружная резьба, 1", никелированная латунь',
 19: 'Диск алмазный по керамограниту Maxx, 125х22.2 мм',
 20: 'Шина соединительная 1Р 63 А',
 21: 'МУФТА АКС PROAQUA 20X1/2 НР',
 22: 'Система выра

In [51]:
def visualisation(cluster, dists, product_dict=None, top=10):
    nodes = []
    edges = []
    edge_labels = dict()
    cluster = cluster[:top]
    
    
    a, b = 0.2, 0.7
    
    def dist_between_products(product1, product2):
        if product1 == product2:
            return 0

        if (product1, product2) in dists:
            return dists[(product1, product2)]

        if (product2, product1) in dists:
            return dists[(product2, product1)]

        return float('inf')
    
    def get_dist_between(product, cluster):
        dist = 0.0
        cnt = 0
        for c in cluster:
            if (product, c) in dists:
                dist += dists[(product, c)] ** 2
                cnt += 1
            elif (c, product) in dists:
                dist += dists[(c, product)] ** 2
                cnt += 1
        if cnt == 0:
            return float('inf')
        
        return dist / cnt
    
    nodes = cluster
    
    
    for p1 in cluster:
        for p2 in cluster:
            if p1 != p2:
                di = dist_between_products(p1, p2)
                if di < float('inf'):
                    edges.append((p1, p2, di))
                    edge_labels[(p1, p2)] = "%.1f" % di
                    
    fig = plt.figure(figsize=(20, 10))
    
    weights = np.array([e[2] for e in edges])
    norm = np.linalg.norm(weights)
    node_color = {
#         cluster[0] : 'tab:blue',
    }   
    
    edge_length = dict()
    for e in edges:
        if (e[0], e[1]) not in edge_length:
            length = e[2] / norm # [0, 1]
            length = length * (b - a) + a # [0.3, 0.7]
            edge_length[(e[0], e[1])] = length
            edge_length[(e[1], e[0])] = length
        e = (e[0], e[1], (1 - e[2] / norm) * 30 + 1)
    
        if e[0] not in node_color:
            node_color[e[0]] = 'white'
        
        if e[1] not in node_color:
            node_color[e[1]] = 'white'
#     print(edge_length)
            
    if len(edge_length.keys()) == 0:
        return
    for p1 in set(np.concatenate(list(edge_length.keys()))):
        for p2 in set(np.concatenate(list(edge_length.keys()))):
            if (p1, p2) not in edge_length:
                edge_length[(p1, p2)] = a
                edge_length[(p2, p1)] = a
        
    
    if product_dict is not None:
        node_labels = dict()
        
        for n in nodes:
            node_labels[n] = product_dict[n]
            
        print(node_labels)
            
#         g = Graph(edges,
#                   node_color=node_color,
#                   node_layout='geometric',
#                   node_layout_kwargs=dict(edge_length=edge_length, tol=1e-3),
#                   node_size=5,
#                   node_label_fontdict={'family': 'serif',
#                                        'weight': 'normal',
#                                        'size': 16,
#                                       },
# #                   node_label_offset=0.05,
#                   node_alpha=0.4,
#                   edge_labels=edge_labels,
#                   node_labels=True,
# #                   node_labels=node_labels,
#                  )
    else:
        g = Graph(edges,
                  node_color=node_color,
                  node_layout='geometric',
                  node_layout_kwargs=dict(edge_length=edge_length, tol=1e-3),
                  node_size=5,
                  node_labels=True,
                  edge_labels=edge_labels,
                 )
    plt.show()

In [10]:
with open(CLUSTERS_PATH + 'k_means.pkl', 'rb') as f:
    clusters = pickle.load(f)
    
with open(CLUSTERS_PATH + 'ward.pkl', 'rb') as f:
    clusters_ward = pickle.load(f)

In [39]:
len(clusters_ward)

10

In [52]:
for c in sorted(clusters_ward, key=(lambda x: len(x)), reverse=True)[0:20]:
    visualisation(c, dists, product_dict=names, top=5)

{1: 'Шпаклёвка полимерная суперфинишная Knauf Ротбанд Паста 18 кг', 16: 'Пескобетон Axton 30 кг', 23: 'Канал плоский Equation 55х110 мм, 1 м', 34: 'Колено соединительное плоское-круглое Equation 55х110 мм D10', 90: 'Пленка защитная 500х400 см 3 шт.'}
{112: 'Бур по бетону SDS-plus 6х42х110 мм Dexter FD-A0107', 390: 'Комплект уголков мебельных с шурупами цвет белый, 6 шт.', 480: 'Заглушка для штанги НСХ ø2.5 см пластик цвет чёрный 2 шт', 481: 'Рельс несущий НСХ 5x120x0.9 см сталь цвет чёрный', 482: 'Держатель для штанги НСХ 7.5x3.6x4.5 см сталь цвет чёрный 2'}
{98: 'Заглушка для подоконника ПВХ двухсторонняя 600 мм', 154: 'Unnamed', 710: 'Валик для водных красок Dexter 180 мм', 727: 'Unnamed', 793: 'Отвод Ø 110х87°полипропилен'}
{61: 'Перчатки хлопчатобумажные обливные размер единый утепленные', 148: 'Unnamed', 165: 'Лента клейкая упаковочная Axton 48 мм x 66 м прозрачная', 292: 'Unnamed', 295: 'Кисть для красок Сибртех 50 мм'}
{9: 'Заглушка Ø 50 мм полипропилен', 15: 'Отвод Ø 50х87° пол

/tmp/ipykernel_19395/3253426179.py:59: RuntimeWarning: invalid value encountered in scalar divide
  length = e[2] / norm # [0, 1]
/tmp/ipykernel_19395/3253426179.py:63: RuntimeWarning: invalid value encountered in scalar divide
  e = (e[0], e[1], (1 - e[2] / norm) * 30 + 1)


In [58]:
for c in clusters:
    c = c[:10]
    for p in c:
        print(names[p])
    print()

Стремянка стальная Zalger с ковриком 4 ступени
НАПРЕССОВОЧНАЯ ГИЛЬЗА РОСТЕРМPPSU 20Х2.8
БУР SDS PLUS 14 Х 460ММ DEXTER
Угол перфорированный ПВХ с сеткой 100х150x3000 мм
Кисть для радиатора 35 мм
ТРУБА REHAU RAUTITAN STABIL 20Х2.9ММ 1М
Пробка Equation, наружная резьба, 1", никелированная латунь
МУФТА АКС PROAQUA 20X1/2 НР
Канал плоский Equation 55х110 мм, 1 м
Кисть флейцевая Matrix «Водные краски» 50 мм

Пластина настенная с соединителем для круглых воздуховодов E
ПЕТЛЯ СЪЁМНАЯ ПРАВАЯ 85Х65Х2 БЕЛЫЙ
Unnamed
ГАЛЕТА С ЗАВЯЗКАМИ 38Х38, СЕРЫЙ
ДВЕРКА ЖАЛЮЗИЙНАЯ 1995Х594Х20ММ СОРТ А

ДУШ. СТОЙКА СО СМЕС-ЛЕМ REDONDO SENSEA
КРЕПЕЖНЫЙ УГОЛ РАВ/СТ KUR-80Х100Х100Х1.8
АНТИСЕПТИК НЕВЫМЫВ ХМФ БФ 30 ЛЕТ, 10Л
Угол крепежный равносторонний KUR 100x100x100x1.8 оцинкованн
Пластина соединительная PS 200x100x1.8
Ламинат «Ель Стокгольм» 33 класс толщина 8 мм 2.153 м²

СР-ВО УНИВЕРС Д/САНТЕХНИКИ UNICUM, 500МЛ
Бюгель для валиков Dexter 110 мм ø6 мм
Бюгель для валиков Dexter 180 мм, ø8 мм
Валик для водных красок

In [4]:
import sys

a = []
print(sys.getsizeof(a))
a.extend([0 for i in range(1000000)])
print(sys.getsizeof(a))

56
8000056


In [1]:
locals().items()

dict_items([('__name__', '__main__'), ('__doc__', 'Automatically created module for IPython interactive environment'), ('__package__', None), ('__loader__', None), ('__spec__', None), ('__builtin__', <module 'builtins' (built-in)>), ('__builtins__', <module 'builtins' (built-in)>), ('_ih', ['', 'locals().items()']), ('_oh', {}), ('_dh', [PosixPath('/home/fanat/Projects/vkr')]), ('In', ['', 'locals().items()']), ('Out', {}), ('get_ipython', <bound method InteractiveShell.get_ipython of <ipykernel.zmqshell.ZMQInteractiveShell object at 0x7fd518afec50>>), ('exit', <IPython.core.autocall.ZMQExitAutocall object at 0x7fd518aff550>), ('quit', <IPython.core.autocall.ZMQExitAutocall object at 0x7fd518aff550>), ('open', <function open at 0x7fd519b36290>), ('_', ''), ('__', ''), ('___', ''), ('_i', ''), ('_ii', ''), ('_iii', ''), ('_i1', 'locals().items()')])